In [1]:
import pandas as pd
import numpy as np
import glob
import os


In [ ]:
#Indsæt fil sti 
folder_path = r""

all_files = glob.glob(os.path.join(folder_path, '*.csv'))


df_list = []

for file in all_files:
    try:
        df = pd.read_csv(file)
        df_list.append(df)
    except Exception as e:
        print(f"Failed to load {file}: {e}")

df = pd.concat(df_list, ignore_index=True)


In [ ]:
df['# Timestamp'] = pd.to_datetime(df['# Timestamp'], dayfirst=True)
df = df[df['Type of mobile'] == 'Class A'].reset_index(drop=True)

C:\Users\tobia\AppData\Local\Temp\ipykernel_12372\3316817758.py:1: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['# Timestamp'] = pd.to_datetime(df['# Timestamp'])


In [4]:
print(df['Type of mobile'].unique())


['Class A']


In [5]:
print(df.shape[0])


27753377


In [6]:
df.drop_duplicates(inplace=True)

In [7]:
print(df.shape[0])

17254087


In [8]:
duplicates = df.duplicated()
print(duplicates.sum())


0


In [9]:
# Calculate the range of latitude and longitude for each MMSI
def calculate_coordinate_range(group):
    lat_range = group['Latitude'].max() - group['Latitude'].min()
    lon_range = group['Longitude'].max() - group['Longitude'].min()
    return pd.Series({'Lat_Range': lat_range, 'Lon_Range': lon_range})

# Group by MMSI and calculate latitude and longitude ranges
coordinate_ranges = df.groupby('MMSI').apply(calculate_coordinate_range).reset_index()

# Merge the result back to the original DataFrame
df = df.merge(coordinate_ranges, on='MMSI', how='left')

# Define thresholds for minimum movement in latitude and longitude
lat_threshold = 0.01  # Adjust based on your data (e.g., ~1 km)
lon_threshold = 0.01  # Adjust based on your data (e.g., ~1 km)

# Filter out boats that moved less than the thresholds
df = df[(df['Lat_Range'] >= lat_threshold) | (df['Lon_Range'] >= lon_threshold)].reset_index(drop=True)

# Drop the range columns if no longer needed
df.drop(columns=['Lat_Range', 'Lon_Range'], inplace=True)

C:\Users\tobia\AppData\Local\Temp\ipykernel_12372\247424872.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  coordinate_ranges = df.groupby('MMSI').apply(calculate_coordinate_range).reset_index()


In [10]:
#hvis vi får lyst til og afgrænse data til et bestemt område, kan vi gøre det her


# min_lat = 55.8    # syd
# max_lat = 58.2    # nord
# min_lon = 10.5    # west
# max_lon = 12.0    # øst


# df = df[
#     (df['Latitude'] >= min_lat) & (df['Latitude'] <= max_lat) &
#     (df['Longitude'] >= min_lon) & (df['Longitude'] <= max_lon)
# ].reset_index(drop=True)


In [11]:
print(df.columns)

Index(['# Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude',
       'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO',
       'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length',
       'Type of position fixing device', 'Draught', 'Destination', 'ETA',
       'Data source type', 'A', 'B', 'C', 'D'],
      dtype='object')


In [12]:
df = df.sort_values(by=['MMSI', '# Timestamp']).reset_index(drop=True)


In [ ]:
#indsæt fil sti til output af ny samlet fil

df.to_csv(, index=False)



In [14]:
print(df.shape[0])

13125445
